# This notebook will aim to synthesize a collaborative filtering recommendation system dataset containing over 7000 users and using the 4000+ jurisprudence docs as items

In [41]:
from utils.data_loaders import load_juris_meta

import numpy as np
import pandas as pd
import time

from concurrent.futures import ThreadPoolExecutor

In [42]:
labor_related_docs_meta = load_juris_meta('./labor_related_docs_meta.csv')
labor_related_docs_meta

,abs_file_path,answer,year_range,title,file_name,year,month,day,gr_number,division,case_code,id
0,d:/Projects/To Github/LaRJ-Corpus/raw jurispru...,LABOR RELATED,1901-1920,"TATSUSABURO YEGAWA, PLAINTIFF AND APPELLEE, VS...",126340044732358.html,1906,11,22,g.r. no. 3388,NaN,6 phil. 750,1
1,d:/Projects/To Github/LaRJ-Corpus/raw jurispru...,LABOR RELATED,1901-1920,"T. SUGO AND K. SHIBATA, PLAINTIFFS AND APPELLE...",12634004479921.html,1906,11,22,g.r. no. 3387,NaN,6 phil. 744,2
2,d:/Projects/To Github/LaRJ-Corpus/raw jurispru...,LABOR RELATED,1901-1920,"THE UNITED STATES, COMPLAINANT AND APPELLEE, V...",12637977142035130542.html,1904,4,9,g.r. no. 1559,NaN,3 phil. 630,3
3,d:/Projects/To Github/LaRJ-Corpus/raw jurispru...,LABOR RELATED,1901-1920,"THE UNITED STATES, COMPLAINANT AND APPELLEE, V...",1263862677733972151.html,1904,4,9,g.r. no. 1585,NaN,3 phil. 631,4
4,d:/Projects/To Github/LaRJ-Corpus/raw jurispru...,LABOR RELATED,1901-1920,"THE UNITED STATES, COMPLAINANT AND APPELLEE, V...",12638653111436063074.html,1904,4,9,g.r. no. 1586,NaN,3 phil. 633,5
...,...,...,...,...,...,...,...,...,...,...,...,...
4223,d:/Projects/To Github/LaRJ-Corpus/raw jurispru...,LABOR RELATED,2020,RINGO B. DAYOWAN TRANSPORT SERVICES OR RINGO B...,16151712921004948288.html,2020,11,10,g.r. no. 226409,FIRST DIVISION,NaN,4224
4224,d:/Projects/To Github/LaRJ-Corpus/raw jurispru...,LABOR RELATED,2020,MINA C. NACILLA AND THE LATE ROBERTO C. JACOBE...,1615183910987321937.html,2020,11,10,g.r. no. 223449,FIRST DIVISION,NaN,4225
4225,d:/Projects/To Github/LaRJ-Corpus/raw jurispru...,LABOR RELATED,2020,"ALCID C. BALBARINO (NOW DECEASED), SUBSTITUTED...",1615948352512956378.html,2020,9,21,g.r. no. 201580,THIRD DIVISION,NaN,4226
4226,d:/Projects/To Github/LaRJ-Corpus/raw jurispru...,LABOR RELATED,2020,"TEAM PACIFIC CORPORATION, FEDERICO M. FERNANDE...",1615955399598086487.html,2020,7,15,g.r. no. 206789,THIRD DIVISION,NaN,4227


In [43]:
labor_related_docs_meta.dtypes

abs_file_path    object
answer           object
year_range       object
title            object
file_name        object
year              int64
month             int64
day               int64
gr_number        object
division         object
case_code        object
id                int64
dtype: object

In [44]:
# # use this for in order to replicate same random numbers turn it off
# # should synthesis of dataset commence
# seed = round(time.time())
# np.random.seed(seed)

# pseudo hyper params
# the range in which the random number of docs are to be generated from for each user
# e.g. user 1 is generated 40 docs to rate, user 20 is generated 10 docs to rate, so on
min_num_of_docs_to_rate = 10
max_num_of_docs_to_rate = 80

# # high arg of np.random.randint is exclusive that is why we add one to 
# # 80 to ensure numbers generated range from 10 to 80 and not 10 to 79 only
# num_of_docs_to_rate = np.random.randint(min_num_of_docs_to_rate, max_num_of_docs_to_rate + 1, 1, dtype=np.int32)[0]
# print(f"num of docs to rate {num_of_docs_to_rate}")

In [45]:
# seed = round(time.time())
# np.random.seed(seed)

# # use 0 to 4227 as the range of id's to generate the docs for each user
# rand_idx = np.random.choice(labor_related_docs_meta.index, size=num_of_docs_to_rate)

# random_items = labor_related_docs_meta.iloc[rand_idx]
# print(f"random_items size: {random_items.shape[0]}")
# random_items

In [46]:
labor_related_docs_meta.loc[:, 'id']

0          1
1          2
2          3
3          4
4          5
        ... 
4223    4224
4224    4225
4225    4226
4226    4227
4227    4228
Name: id, Length: 4228, dtype: int64

In [47]:
def helper(user):
    """

    """
    num_of_docs_to_rate = np.random.randint(min_num_of_docs_to_rate, max_num_of_docs_to_rate + 1, 1, dtype=np.int32)[0]

    rand_indeces = np.random.choice(labor_related_docs_meta.index, size=num_of_docs_to_rate)
    random_items = labor_related_docs_meta.loc[rand_indeces, 'id'].tolist()

    random_ratings = np.random.randint(1, 5 + 1, size=num_of_docs_to_rate, dtype=np.int32).tolist()
    
    return [random_items, random_ratings]

In [48]:
with ThreadPoolExecutor() as executor:
    results = list(executor.map(helper, np.arange(7000) + 1))

In [49]:
results

[[[3571,
   3235,
   3940,
   4182,
   4186,
   3805,
   2999,
   1667,
   3248,
   2066,
   1343,
   4064,
   1801,
   3726,
   2815,
   3814,
   3439,
   1627,
   3885,
   3386,
   563,
   2117,
   2040,
   3028,
   3354,
   4174,
   1734,
   1341,
   196,
   1793,
   4106,
   352,
   4096,
   3460,
   1160,
   1475,
   36,
   1466,
   2270,
   2450,
   691,
   1402,
   1808,
   2818,
   1254,
   2121,
   1125,
   2257,
   1062,
   3329,
   2102,
   2541,
   2973,
   2433,
   1139,
   3308,
   644,
   3250,
   2089,
   2501,
   2714,
   665,
   3831,
   854,
   2156,
   3811,
   554,
   3535,
   1100],
  [5,
   4,
   5,
   4,
   1,
   4,
   1,
   1,
   1,
   3,
   2,
   3,
   3,
   4,
   4,
   2,
   3,
   5,
   2,
   2,
   2,
   2,
   1,
   1,
   2,
   2,
   3,
   4,
   4,
   2,
   4,
   5,
   5,
   4,
   5,
   2,
   2,
   5,
   1,
   4,
   2,
   1,
   3,
   5,
   3,
   3,
   2,
   2,
   3,
   2,
   5,
   4,
   1,
   5,
   1,
   2,
   4,
   2,
   3,
   3,
   4,
   5,
   4,
   2,
   4